In [1]:
import numpy as np
import pandas as pd
from scipy.stats import spearmanr, boxcox, pearsonr, ks_2samp

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.impute import KNNImputer
from sklearn.linear_model import Lasso

import json
import os
os.chdir("..")

def metric_train(output, truth):
    return spearmanr(output, truth).correlation

In [2]:
EXOTIC_PATH = "data/processed/exotic"
NON_EXOTIC_PATH = "data/processed/non_exotic"
FULL_PATH = "data/processed/full"

In [3]:
X_train = pd.read_csv("data/raw/X_train_NHkHMNU.csv").set_index("ID")
y_train = pd.read_csv("data/raw/y_train_ZAN5mwg.csv").set_index("ID")
X_test = pd.read_csv("data/raw/X_test_final.csv").set_index("ID")
X_train.shape, y_train.shape, X_test.shape

((1494, 34), (1494, 1), (654, 34))

In [4]:
country_code = {"FR": 0, "DE": 1}

full_train = y_train.join(X_train)
full_train = full_train.replace(country_code)

full_train["RANK"] = full_train["TARGET"].rank()

X_test = X_test.replace(country_code)

In [5]:
def split_exotic_idx(df: pd.DataFrame):
    day_count = df.groupby("DAY_ID")["COUNTRY"].count()
    exotic_day = day_count[day_count == 1].index.to_list()
    non_exotic_day = day_count[day_count == 2].index.to_list()
    return exotic_day, non_exotic_day

In [6]:
exotic_train_idx, non_exotic_train_idx = split_exotic_idx(full_train)
exotic_train_df = full_train.loc[full_train["DAY_ID"].isin(exotic_train_idx)]
non_exotic_train_df = full_train.loc[full_train["DAY_ID"].isin(non_exotic_train_idx)]

In [7]:
exotic_test, non_exotic_test = split_exotic_idx(X_test)
exotic_test_X = X_test.loc[X_test["DAY_ID"].isin(exotic_test)]
non_exotic_test_X = X_test.loc[X_test["DAY_ID"].isin(non_exotic_test)]

In [8]:
print(exotic_train_df.columns[exotic_train_df.isna().any(axis=0) != 0])
print(non_exotic_train_df.columns[non_exotic_train_df.isna().any(axis=0) != 0])
print(exotic_test_X.columns[exotic_test_X.isna().any(axis=0) != 0])
print(non_exotic_test_X.columns[non_exotic_test_X.isna().any(axis=0) != 0])

Index(['DE_FR_EXCHANGE', 'FR_DE_EXCHANGE', 'DE_NET_EXPORT', 'FR_NET_EXPORT',
       'DE_NET_IMPORT', 'FR_NET_IMPORT'],
      dtype='object')
Index(['DE_RAIN', 'FR_RAIN', 'DE_WIND', 'FR_WIND', 'DE_TEMP', 'FR_TEMP'], dtype='object')
Index(['DE_FR_EXCHANGE', 'FR_DE_EXCHANGE', 'DE_NET_EXPORT', 'FR_NET_EXPORT',
       'DE_NET_IMPORT', 'FR_NET_IMPORT'],
      dtype='object')
Index(['DE_RAIN', 'FR_RAIN', 'DE_WIND', 'FR_WIND', 'DE_TEMP', 'FR_TEMP'], dtype='object')


In [9]:
# Median fillna
exotic_train_df = exotic_train_df.fillna(exotic_train_df.median())
exotic_test_X = exotic_test_X.fillna(exotic_test_X.median())

def fill_median_by_country(df: pd.DataFrame, df_fit: pd.DataFrame):
    df = df.copy()
    median_filler = df_fit.loc[df_fit.COUNTRY == 0].median()
    df.loc[df.COUNTRY == 0] = df.loc[df.COUNTRY == 0].fillna(median_filler)
    median_filler = df_fit.loc[df_fit.COUNTRY == 1].median()
    df.loc[df.COUNTRY == 1] = df.loc[df.COUNTRY == 1].fillna(median_filler)
    return df

non_exotic_train_df = fill_median_by_country(non_exotic_train_df, non_exotic_train_df)
non_exotic_test_X = fill_median_by_country(non_exotic_test_X, non_exotic_train_df)

full_train = fill_median_by_country(full_train, full_train)
X_test = fill_median_by_country(X_test, full_train)

In [10]:
IMPUTE_STAGE = "median_imputed"
# exotic_train_df.to_csv(f"{EXOTIC_PATH}/{IMPUTE_STAGE}_train.csv")
# exotic_test_X.to_csv(f"{EXOTIC_PATH}/{IMPUTE_STAGE}_test.csv")
# non_exotic_train_df.to_csv(f"{NON_EXOTIC_PATH}/{IMPUTE_STAGE}_train.csv")
# non_exotic_test_X.to_csv(f"{NON_EXOTIC_PATH}/{IMPUTE_STAGE}_test.csv")
# full_train.to_csv(f"{FULL_PATH}/{IMPUTE_STAGE}_train.csv")
# X_test.to_csv(f"{FULL_PATH}/{IMPUTE_STAGE}_test.csv")

In [11]:
non_exotic_train_df_fr = non_exotic_train_df.loc[non_exotic_train_df.COUNTRY == 0]
non_exotic_train_df_de = non_exotic_train_df.loc[non_exotic_train_df.COUNTRY == 1]
# non_exotic_train_df_fr.to_csv(f"{NON_EXOTIC_PATH}/{IMPUTE_STAGE}_train_fr.csv")
# non_exotic_train_df_de.to_csv(f"{NON_EXOTIC_PATH}/{IMPUTE_STAGE}_train_de.csv")

non_exotic_test_X_fr = non_exotic_test_X.loc[non_exotic_test_X.COUNTRY == 0]
non_exotic_test_X_de = non_exotic_test_X.loc[non_exotic_test_X.COUNTRY == 1]
# non_exotic_test_X_fr.to_csv(f"{NON_EXOTIC_PATH}/{IMPUTE_STAGE}_test_fr.csv")
# non_exotic_test_X_de.to_csv(f"{NON_EXOTIC_PATH}/{IMPUTE_STAGE}_test_de.csv")

In [12]:
feature_cols = X_train.columns.drop(["DAY_ID", "COUNTRY"]).to_list()
len(feature_cols)

32

In [13]:
def different_dist(df1: pd.DataFrame, df2: pd.DataFrame, pvalue=0.05):
    return [col for col in feature_cols if ks_2samp(df1[col], df2[col]).pvalue < pvalue]

In [14]:
# Non-exotic X's are same dist
ks_non_exotic = different_dist(non_exotic_train_df.loc[non_exotic_train_df.COUNTRY == 0], non_exotic_train_df.loc[non_exotic_train_df.COUNTRY == 1])
print(len(ks_non_exotic))
# Exotic X's are different dist from non-exotic X's
ks_exotic = different_dist(non_exotic_train_df, exotic_train_df)
ks_fr_exotic = different_dist(non_exotic_train_df.loc[non_exotic_train_df.COUNTRY == 0], exotic_train_df)
ks_de_exotic = different_dist(non_exotic_train_df.loc[non_exotic_train_df.COUNTRY == 1], exotic_train_df)
print(len(ks_exotic), len(ks_fr_exotic), len(ks_de_exotic))
# Non-exotic targets are different distributions by country
print(
    ks_2samp(
        non_exotic_train_df.loc[non_exotic_train_df.COUNTRY == 0, "TARGET"], 
        non_exotic_train_df.loc[non_exotic_train_df.COUNTRY == 1, "TARGET"],
    ).pvalue
)
# exotic targets, same distribution as non exotic FR but not DE
print(
    ks_2samp(
        non_exotic_train_df.loc[non_exotic_train_df.COUNTRY == 0, "TARGET"], 
        exotic_train_df["TARGET"],
    ).pvalue
)
print(
    ks_2samp(
        non_exotic_train_df.loc[non_exotic_train_df.COUNTRY == 1, "TARGET"], 
        exotic_train_df["TARGET"],
    ).pvalue
)

0
25 25 25
5.63222408650964e-05
0.5002043840391661
0.0002135096019779681


In [15]:
# common_feature_cols = list(set(feature_cols).difference(ks_exotic))

exotic_drift_cols = different_dist(exotic_train_df, exotic_test_X)
non_exotic_drift_cols = different_dist(non_exotic_train_df, non_exotic_test_X)
exotic_drift_cols, non_exotic_drift_cols

(['DE_NET_EXPORT', 'FR_NET_EXPORT', 'DE_NET_IMPORT', 'FR_NET_IMPORT'],
 ['DE_CONSUMPTION',
  'FR_CONSUMPTION',
  'DE_FR_EXCHANGE',
  'FR_DE_EXCHANGE',
  'FR_NET_EXPORT',
  'FR_NET_IMPORT',
  'FR_NUCLEAR',
  'DE_LIGNITE',
  'FR_RESIDUAL_LOAD',
  'FR_WIND',
  'DE_TEMP',
  'FR_TEMP'])

In [16]:
class feat_selector():
    def __init__(self, corr, target="TARGET"):
        self.corr = corr
        self.target = target

    def multi_colinear_feat(self, thres=0.8):
        losers = set()
        
        tri_mask = np.triu(np.ones(self.corr.shape, dtype=np.bool_))
        multi_collin = self.corr[self.corr.mask(tri_mask).abs() > 0.8]
        mc_idxs = np.stack(multi_collin.notnull().values.nonzero()).T.tolist()
        mc_cols = [[self.corr.columns[i1], self.corr.columns[i2]] for (i1, i2) in mc_idxs]

        return [
            c2 if abs(self.corr[self.target][c1]) < abs(self.corr[self.target][c2]) else c1 for c1, c2 in mc_cols
            ]
    
    def weak_feat(self, thres=0.05):
        weak_flag = self.corr[self.target].abs() < thres
        return weak_flag.index[weak_flag].to_list()

In [17]:
non_exotic_remainder = list(set(feature_cols).difference(non_exotic_drift_cols))

tmp_corr = non_exotic_train_df_fr[["TARGET"]+non_exotic_remainder].corr("spearman")
non_exotic_fr_fs = feat_selector(tmp_corr)
non_exotic_fr_drop_cols = list(set(non_exotic_fr_fs.multi_colinear_feat() + non_exotic_fr_fs.weak_feat()))
non_exotic_fr_feature_cols = list(set(non_exotic_remainder).difference(non_exotic_fr_drop_cols))

tmp_corr["TARGET"][non_exotic_fr_feature_cols]

DE_WINDPOW         -0.079221
DE_NET_IMPORT       0.052830
FR_RAIN            -0.068064
FR_HYDRO            0.090760
DE_HYDRO            0.111398
CARBON_RET          0.185105
FR_COAL             0.096489
DE_RESIDUAL_LOAD    0.066694
FR_WINDPOW         -0.112430
GAS_RET             0.151401
DE_GAS              0.056043
DE_RAIN            -0.063261
Name: TARGET, dtype: float64

In [18]:
tmp_corr = non_exotic_train_df_de[["TARGET"]+non_exotic_remainder].corr("spearman")
non_exotic_de_fs = feat_selector(tmp_corr)
non_exotic_de_drop_cols = list(set(non_exotic_de_fs.multi_colinear_feat() + non_exotic_de_fs.weak_feat()))
non_exotic_de_feature_cols = list(set(non_exotic_remainder).difference(non_exotic_de_drop_cols))

tmp_corr["TARGET"][non_exotic_de_feature_cols]

DE_WINDPOW         -0.300933
DE_WIND            -0.147871
DE_COAL             0.142054
DE_NET_IMPORT       0.306204
FR_RAIN            -0.075194
FR_HYDRO            0.058362
DE_HYDRO            0.217900
DE_RESIDUAL_LOAD    0.324335
FR_WINDPOW         -0.199820
DE_GAS              0.253410
FR_GAS              0.072784
Name: TARGET, dtype: float64

In [19]:
exotic_remainder = list(set(feature_cols).difference(exotic_drift_cols))

tmp_corr = exotic_train_df[["TARGET"]+exotic_remainder].corr("spearman")
exotic_fs = feat_selector(tmp_corr)
exotic_drop_cols = list(set(exotic_fs.multi_colinear_feat() + exotic_fs.weak_feat()))
exotic_feature_cols = list(set(exotic_remainder).difference(exotic_drop_cols))

tmp_corr["TARGET"][exotic_feature_cols]

DE_WINDPOW       -0.090380
FR_RAIN           0.113682
FR_HYDRO          0.062984
DE_FR_EXCHANGE   -0.114560
COAL_RET          0.192784
DE_NUCLEAR       -0.064955
FR_SOLAR          0.081600
DE_SOLAR          0.155697
CARBON_RET        0.207799
DE_LIGNITE        0.064084
FR_WINDPOW       -0.157108
GAS_RET           0.141679
FR_GAS           -0.086897
Name: TARGET, dtype: float64

In [20]:
feature_selection = {
    "de": non_exotic_de_feature_cols,
    "fr": non_exotic_fr_feature_cols,
    "exotic": exotic_feature_cols,
}

In [21]:
# json.dump(feature_selection, open("features/feature_selection.json", "w"))